In [715]:
import pandas as pd
import numpy as np 
from github import Github
import getpass
from functools import reduce
import math
from operator import itemgetter, attrgetter

In [716]:
# g = Github('WellersonPrenholato', getpass.getpass('Senha:'))

In [717]:
usedLanguages={}
for repo in g.get_user().get_repos():
    repoLangs = repo.get_languages()
    for lang in repoLangs:
        if lang in usedLanguages:
            usedLanguages[lang] += repoLangs[lang]
        else:
            usedLanguages[lang] = repoLangs[lang]

# print(usedLanguages)

{'Java': 141020, 'TypeScript': 191454, 'HTML': 9031127, 'JavaScript': 1607891, 'CSS': 230576, 'PHP': 5975421, 'Jupyter Notebook': 6751266, 'C++': 159910, 'Python': 1914514, 'C': 954738, 'Shell': 7750, 'TeX': 125055, 'Makefile': 4006, 'Assembly': 4796, 'Processing': 23061, 'Cuda': 72651, 'Go': 3330, 'C#': 3211, 'Kotlin': 2346, 'Brainfuck': 1836, 'Batchfile': 1502, 'Ruby': 1493, 'Swift': 1312, 'Elixir': 805, 'Brightscript': 701, 'R': 684, 'Visual Basic': 432, 'Pascal': 430, 'PowerShell': 429, 'Rust': 387, 'Scala': 377, 'MATLAB': 351, 'Haskell': 348, 'Lua': 342, 'Elm': 324, 'Pike': 324, 'Apex': 314, 'LOLCODE': 311, 'Hack': 269, 'Scheme': 257, 'D': 255, 'Tcl': 228, 'F#': 225, 'Nemerle': 221, 'Fortran': 217, 'Crystal': 211, 'Common Lisp': 195, 'Ada': 192, 'xBase': 165, 'Dart': 137, 'Prolog': 134, 'Haxe': 133, 'Perl 6': 130, 'ABAP': 125, 'COBOL': 123, 'OCaml': 123, 'CoffeeScript': 122, 'Perl': 120, 'Smalltalk': 114, 'Groovy': 113, 'q': 107, 'Julia': 107, 'Forth': 105, 'Nim': 84, 'Erlang': 80

In [718]:
# total = reduce(lambda x, value:x + value, usedLanguages.values(), 0)
# total

27221329

In [719]:
#Talvez transformar o valor incial encontrado em um dataframe e depois realizar os cálculos para mudança de porcentagem

In [720]:
conjLinguagens = [];
pesoLinguagens = [];
for linguagem, peso in zip(list(usedLanguages.keys()), list(usedLanguages.values())):
    conjLinguagens.append(linguagem)
    pesoLinguagens.append(peso)

In [721]:
armazena_data = {'Linguagens': conjLinguagens, 'Peso': pesoLinguagens};

In [722]:
#Transforma o conjunto de listas em um dataframe
data = pd.DataFrame(armazena_data)

In [723]:
#Gera o arquivo csv com apenas os atributos selecionados pela árvore de decisãob
data.to_csv('Linguagens.csv', encoding= 'ISO-8859-1', sep= ';', index=False)

In [724]:
data.sort_values(by=['Peso'], ascending=False, inplace=True)

In [725]:
# data.head(20)

In [726]:
data = data.drop(data[(data.Linguagens == 'HTML')].index)

In [727]:
data = data.drop(data[(data.Linguagens == 'CSS')].index)

In [728]:
#Gera o arquivo csv com apenas os atributos selecionados pela árvore de decisãob
data.to_csv('Linguagens.csv', encoding= 'ISO-8859-1', sep= ';', index=False)

In [729]:
data = pd.read_csv('Linguagens.csv', encoding='ISO-8859-1', sep=';')

In [730]:
qtdLanguages = 11
indices=[x for x in data.index if (x >= qtdLanguages)]
data.drop(indices, inplace=True)

In [731]:
data.head(20)

,Linguagens,Peso
0,Jupyter Notebook,6751266
1,PHP,5975421
2,Python,1914514
3,JavaScript,1607891
4,C,954738
5,TypeScript,191454
6,C++,159910
7,Java,141020
8,TeX,125055
9,Cuda,72651


In [732]:
total = data['Peso'].sum()
# total = reduce(lambda x, value: x + value, data['Peso'], 0)
total

17916981

In [733]:
data['Peso'] = pd.to_numeric(data['Peso'], downcast='float')

In [734]:
data.dtypes

Linguagens     object
Peso          float32
dtype: object

In [735]:
# data['Peso'].sum()

In [736]:
for index, row in data.iterrows():
    data.at[index, 'Peso'] /= total
    data.at[index, 'Peso'] = round(data.at[index, 'Peso'] * 100, 3)
    print(data.at[index, 'Peso'])

37.681
33.351
10.685
8.974
5.329
1.069
0.893
0.787
0.698
0.405
0.129


In [737]:
data.head(10)

,Linguagens,Peso
0,Jupyter Notebook,37.681000
1,PHP,33.351002
2,Python,10.685000
3,JavaScript,8.974000
4,C,5.329000
5,TypeScript,1.069000
6,C++,0.893000
7,Java,0.787000
8,TeX,0.698000
9,Cuda,0.405000


In [738]:

# total_acertos_por_questao = pd.DataFrame(matriz_acertos.sum())
# total_acertos_por_questao.reset_index(inplace=True)
# total_acertos_por_questao.columns = ['Questão', 'Número de Acertos']
# total_acertos_por_questao['Questão'] = total_acertos_por_questao['Questão'] + 1

# plt.figure(figsize=(18, 8))
# ax = sns.barplot(x='Questão', y='Número de Acertos', data=total_acertos_por_questao)

# ax.set_title('Linguagens', fontsize=16)

# plt.show()